In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

train_data_dir = Path('../data/original/train.csv')
test_data_dir = Path('../data/original/test.csv')
stores_data_dir = Path('../data/original/stores.csv')
transactions_data_dir = Path('../data/original/transactions.csv')
holidays_events_data_dir = Path('../data/original/holidays_events.csv')
oil_data_dir = Path('../data/original/oil.csv')


training_datatype_map = {
    'store_nbr': 'uint8',
    'family': 'category',
    'sales': 'float32',
    'onpromotion': 'uint64',
}
testing_datatype_map = {
    'store_nbr': 'uint8',
    'family': 'category',
    'onpromotion': 'uint64',
}
stores_datatype_map = {
    'cluster': 'int8',
}

train_df = pd.read_csv(
    train_data_dir,
    dtype=training_datatype_map,
    parse_dates=['date'],
)
test_df = pd.read_csv(
    test_data_dir,
    dtype=testing_datatype_map,
    parse_dates=['date'],
)
stores_df = pd.read_csv(stores_data_dir, dtype=stores_datatype_map)   
transactions_df = pd.read_csv(transactions_data_dir, parse_dates=['date']).sort_values(['date', 'store_nbr'])
holidays_events_df = pd.read_csv(holidays_events_data_dir, parse_dates=['date'])
oil_df = pd.read_csv(oil_data_dir, parse_dates=['date'])

In [69]:
from darts.timeseries import TimeSeries
import sklearn.preprocessing as preprocessing
from darts.utils.missing_values import fill_missing_values
sdf = train_df.loc[(train_df.store_nbr == 1) & (train_df.family == 'GROCERY I')]

events_df = pd.read_csv(
            holidays_events_data_dir,
            parse_dates=['date'],
        )
events_df = events_df.loc[
    (events_df.transferred == False) &
    ((events_df.type == 'Holiday') |
        (events_df.type == 'Additional') |
        (events_df.type == 'Bridge')) &
    (events_df.locale != 'Local')
]
events_df = events_df.drop_duplicates(subset='date')
events_df = events_df[['date', 'type']]

sdf = sdf.join(
            events_df.set_index('date'), on='date', validate='m:1'
        )

le = preprocessing.LabelEncoder()
sdf['type'] = le.fit_transform(sdf['type'])

data = TimeSeries.from_dataframe(
        sdf,
        time_col='date',
        value_cols='sales',
        static_covariates=sdf['type'],
        fill_missing_dates=True,
        freq='D',
    )

data = fill_missing_values(data)
len(sdf.index)
data.static_covariates

static_covariates,12,1794,3576,5358,7140,8922,10704,12486,14268,16050,17832,19614,21396,23178,24960,26742,28524,30306,32088,33870,35652,37434,39216,40998,42780,44562,46344,48126,49908,51690,53472,55254,57036,58818,60600,62382,64164,65946,67728,69510,71292,73074,74856,76638,78420,80202,81984,83766,85548,87330,89112,90894,92676,94458,96240,98022,99804,101586,103368,105150,106932,108714,110496,112278,114060,115842,117624,119406,121188,122970,124752,126534,128316,130098,131880,133662,135444,137226,139008,140790,142572,144354,146136,147918,149700,151482,153264,155046,156828,158610,160392,162174,163956,165738,167520,169302,171084,172866,174648,176430,178212,179994,181776,183558,185340,187122,188904,190686,192468,194250,196032,197814,199596,201378,203160,204942,206724,208506,210288,212070,213852,215634,217416,219198,220980,222762,224544,226326,228108,229890,231672,233454,235236,237018,238800,240582,242364,244146,245928,247710,249492,251274,253056,254838,256620,258402,260184,261966,263748,265530,267312,269094,270876,272658,274440,276222,278004,279786,281568,283350,285132,286914,288696,290478,292260,294042,295824,297606,299388,301170,302952,304734,306516,308298,310080,311862,313644,315426,317208,318990,320772,322554,324336,326118,327900,329682,331464,333246,335028,336810,338592,340374,342156,343938,345720,347502,349284,351066,352848,354630,356412,358194,359976,361758,363540,365322,367104,368886,370668,372450,374232,376014,377796,379578,381360,383142,384924,386706,388488,390270,392052,393834,395616,397398,399180,400962,402744,404526,406308,408090,409872,411654,413436,415218,417000,418782,420564,422346,424128,425910,427692,429474,431256,433038,434820,436602,438384,440166,441948,443730,445512,447294,449076,450858,452640,454422,456204,457986,459768,461550,463332,465114,466896,468678,470460,472242,474024,475806,477588,479370,481152,482934,484716,486498,488280,490062,491844,493626,495408,497190,498972,500754,502536,504318,506100,507882,509664,511446,513228,515010,516792,518574,520356,522138,523920,525702,527484,529266,531048,532830,534612,536394,538176,539958,541740,543522,545304,547086,548868,550650,552432,554214,555996,557778,559560,561342,563124,564906,566688,568470,570252,572034,573816,575598,577380,579162,580944,582726,584508,586290,588072,589854,591636,593418,595200,596982,598764,600546,602328,604110,605892,607674,609456,611238,613020,614802,616584,618366,620148,621930,623712,625494,627276,629058,630840,632622,634404,636186,637968,639750,641532,643314,645096,646878,648660,650442,652224,654006,655788,657570,659352,661134,662916,664698,666480,668262,670044,671826,673608,675390,677172,678954,680736,682518,684300,686082,687864,689646,691428,693210,694992,696774,698556,700338,702120,703902,705684,707466,709248,711030,712812,714594,716376,718158,719940,721722,723504,725286,727068,728850,730632,732414,734196,735978,737760,739542,741324,743106,744888,746670,748452,750234,752016,753798,755580,757362,759144,760926,762708,764490,766272,768054,769836,771618,773400,775182,776964,778746,780528,782310,784092,785874,787656,789438,791220,793002,794784,796566,798348,800130,801912,803694,805476,807258,809040,810822,812604,814386,816168,817950,819732,821514,823296,825078,826860,828642,830424,832206,833988,835770,837552,839334,841116,842898,844680,846462,848244,850026,851808,853590,855372,857154,858936,860718,862500,864282,866064,867846,869628,871410,873192,874974,876756,878538,880320,882102,883884,885666,887448,889230,891012,892794,894576,896358,898140,899922,901704,903486,905268,907050,908832,910614,912396,914178,915960,917742,919524,921306,923088,924870,926652,928434,930216,931998,933780,935562,937344,939126,940908,942690,944472,946254,948036,949818,951600,953382,955164,956946,958728,960510,962292,964074,965856,967638,969420,971202,972984,974766,976548,978330,980112,981894,983676,985458,987240,989022,990804,992586,994368,996150,997932,999714,1001496,1003278,1005060,1006842,1008624,1010406,1012188,1013970,1015752,1017534,1019316,1021098,1022880,1024662,1

In [64]:
pd.concat([sdf, None])

,id,date,store_nbr,family,sales,onpromotion,type
12,12,2013-01-01,1,GROCERY I,0.00,0,2
1794,1794,2013-01-02,1,GROCERY I,2652.00,0,3
3576,3576,2013-01-03,1,GROCERY I,2121.00,0,3
5358,5358,2013-01-04,1,GROCERY I,2056.00,0,3
7140,7140,2013-01-05,1,GROCERY I,2216.00,0,3
...,...,...,...,...,...,...,...
2991990,2991990,2017-08-11,1,GROCERY I,1270.00,24,3
2993772,2993772,2017-08-12,1,GROCERY I,1630.00,28,3
2995554,2995554,2017-08-13,1,GROCERY I,952.00,19,3
2997336,2997336,2017-08-14,1,GROCERY I,2407.00,30,3


In [72]:
from darts.models import LightGBMModel, ExponentialSmoothing
from darts.utils.utils import ModelMode, SeasonalityMode

model = LightGBMModel(lags=14, use_static_covariates=True)
a = model.fit(data)
model.predict(10)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000204 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3570
[LightGBM] [Info] Number of data points in the train set: 1674, number of used features: 14
[LightGBM] [Info] Start training from score 2227.901135


<TimeSeries (DataArray) (date: 10, component: 1, sample: 1)>
array([[[2631.90096718]],

       [[2168.16702531]],

       [[2531.36606871]],

       [[2286.35654209]],

       [[ 992.60229768]],

       [[2716.83440071]],

       [[2827.70918332]],

       [[2847.96429955]],

       [[2461.14306056]],

       [[2623.04398063]]])
Coordinates:
  * date       (date) datetime64[ns] 2017-08-16 2017-08-17 ... 2017-08-25
  * component  (component) object 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  12       1794     3576     5358   ...
    hierarchy:          None